# Tableau Dashboard Dataset Creation
This notebook creates a comprehensive, client-friendly dataset optimized for Tableau visualizations to help understand credit risk patterns and insights.

In [3]:
# 1. Import libraries and setup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("📊 TABLEAU DASHBOARD DATA PREPARATION")
print("="*50)
print("Creating client-friendly dataset for credit risk visualization")
print("Target: Single comprehensive CSV for Tableau dashboard")
print("="*50)

📊 TABLEAU DASHBOARD DATA PREPARATION
Creating client-friendly dataset for credit risk visualization
Target: Single comprehensive CSV for Tableau dashboard


In [5]:
# 2. Load main datasets
print("Loading main datasets...")

# Load application train data (main dataset)
try:
    # Try enriched first, fallback to original if too large
    print("Attempting to load enriched dataset...")
    app_train = pd.read_csv('application_train_enriched.csv')
    print(f"✅ Loaded enriched training data: {app_train.shape}")
    dataset_type = "enriched"
except:
    print("Loading original training data...")
    app_train = pd.read_csv('application_train.csv')
    print(f"✅ Loaded original training data: {app_train.shape}")
    dataset_type = "original"

# Load column descriptions for better labeling (handle encoding issues)
try:
    col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='utf-8')
    print(f"✅ Loaded column descriptions: {col_desc.shape}")
except UnicodeDecodeError:
    try:
        col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='latin-1')
        print(f"✅ Loaded column descriptions with latin-1 encoding: {col_desc.shape}")
    except:
        try:
            col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='cp1252')
            print(f"✅ Loaded column descriptions with cp1252 encoding: {col_desc.shape}")
        except:
            print("⚠️ Could not load column descriptions file - proceeding without it")
            col_desc = None

print(f"\nDataset type: {dataset_type}")
print(f"Training data columns: {len(app_train.columns)}")
print(f"Sample size: {len(app_train):,} records")

Loading main datasets...
Attempting to load enriched dataset...
✅ Loaded enriched training data: (307511, 160)
✅ Loaded column descriptions with latin-1 encoding: (219, 5)

Dataset type: enriched
Training data columns: 160
Sample size: 307,511 records
✅ Loaded enriched training data: (307511, 160)
✅ Loaded column descriptions with latin-1 encoding: (219, 5)

Dataset type: enriched
Training data columns: 160
Sample size: 307,511 records


In [6]:
# 3. Create client-friendly categorical mappings
print("Creating client-friendly labels...")

def create_friendly_labels(df):
    """Transform technical codes into human-readable labels"""
    
    tableau_df = df.copy()
    
    # 1. TARGET - Risk Category
    if 'TARGET' in tableau_df.columns:
        tableau_df['Risk_Category'] = tableau_df['TARGET'].map({
            0: 'Low Risk (Good Client)',
            1: 'High Risk (Payment Issues)'
        })
    
    # 2. Gender
    if 'CODE_GENDER' in tableau_df.columns:
        tableau_df['Gender'] = tableau_df['CODE_GENDER'].replace({
            'M': 'Male',
            'F': 'Female',
            'XNA': 'Not Specified'
        })
    
    # 3. Contract Type
    if 'NAME_CONTRACT_TYPE' in tableau_df.columns:
        tableau_df['Loan_Type'] = tableau_df['NAME_CONTRACT_TYPE'].replace({
            'Cash loans': 'Cash Loan',
            'Revolving loans': 'Revolving Credit'
        })
    
    # 4. Income Type
    if 'NAME_INCOME_TYPE' in tableau_df.columns:
        tableau_df['Income_Source'] = tableau_df['NAME_INCOME_TYPE'].replace({
            'Working': 'Employed',
            'Commercial associate': 'Business Associate',
            'Pensioner': 'Retiree',
            'State servant': 'Government Employee',
            'Student': 'Student',
            'Businessman': 'Business Owner',
            'Maternity leave': 'Maternity Leave',
            'Unemployed': 'Unemployed'
        })
    
    # 5. Education
    if 'NAME_EDUCATION_TYPE' in tableau_df.columns:
        tableau_df['Education_Level'] = tableau_df['NAME_EDUCATION_TYPE'].replace({
            'Secondary / secondary special': 'High School',
            'Higher education': 'University',
            'Incomplete higher': 'Some University',
            'Lower secondary': 'Middle School',
            'Academic degree': 'Graduate Degree'
        })
    
    # 6. Family Status
    if 'NAME_FAMILY_STATUS' in tableau_df.columns:
        tableau_df['Marital_Status'] = tableau_df['NAME_FAMILY_STATUS'].replace({
            'Married': 'Married',
            'Single / not married': 'Single',
            'Civil marriage': 'Common Law',
            'Separated': 'Separated',
            'Widow': 'Widowed',
            'Unknown': 'Not Specified'
        })
    
    # 7. Housing Type
    if 'NAME_HOUSING_TYPE' in tableau_df.columns:
        tableau_df['Housing_Type'] = tableau_df['NAME_HOUSING_TYPE'].replace({
            'House / apartment': 'Own Home',
            'With parents': 'Living with Parents',
            'Municipal apartment': 'Public Housing',
            'Rented apartment': 'Rental',
            'Office apartment': 'Office Space',
            'Co-op apartment': 'Cooperative'
        })
    
    return tableau_df

# Apply friendly labels
tableau_data = create_friendly_labels(app_train)
print(f"✅ Created friendly labels for categorical variables")
print(f"New columns added: Risk_Category, Gender, Loan_Type, Income_Source, etc.")

Creating client-friendly labels...
✅ Created friendly labels for categorical variables
New columns added: Risk_Category, Gender, Loan_Type, Income_Source, etc.
✅ Created friendly labels for categorical variables
New columns added: Risk_Category, Gender, Loan_Type, Income_Source, etc.


In [7]:
# 4. Create derived metrics for better insights
print("Creating derived business metrics...")

def create_business_metrics(df):
    """Create meaningful business metrics for dashboard"""
    
    dashboard_df = df.copy()
    
    # 1. Age from DAYS_BIRTH
    if 'DAYS_BIRTH' in dashboard_df.columns:
        dashboard_df['Age_Years'] = abs(dashboard_df['DAYS_BIRTH']) / 365.25
        dashboard_df['Age_Group'] = pd.cut(
            dashboard_df['Age_Years'], 
            bins=[0, 25, 35, 45, 55, 65, 100], 
            labels=['Under 25', '25-34', '35-44', '45-54', '55-64', '65+']
        )
    
    # 2. Employment Duration
    if 'DAYS_EMPLOYED' in dashboard_df.columns:
        # Fix anomaly (365243 is a known data issue)
        dashboard_df['DAYS_EMPLOYED'] = dashboard_df['DAYS_EMPLOYED'].replace(365243, np.nan)
        dashboard_df['Employment_Years'] = abs(dashboard_df['DAYS_EMPLOYED']) / 365.25
        dashboard_df['Employment_Years'] = dashboard_df['Employment_Years'].clip(0, 50)  # Cap at 50 years
        dashboard_df['Employment_Category'] = pd.cut(
            dashboard_df['Employment_Years'], 
            bins=[0, 1, 3, 5, 10, 50], 
            labels=['Less than 1 year', '1-3 years', '3-5 years', '5-10 years', '10+ years']
        )
    
    # 3. Income Categories
    if 'AMT_INCOME_TOTAL' in dashboard_df.columns:
        dashboard_df['Monthly_Income'] = dashboard_df['AMT_INCOME_TOTAL'] / 12
        income_percentiles = dashboard_df['Monthly_Income'].quantile([0.25, 0.5, 0.75])
        dashboard_df['Income_Level'] = pd.cut(
            dashboard_df['Monthly_Income'],
            bins=[0, income_percentiles[0.25], income_percentiles[0.5], 
                  income_percentiles[0.75], dashboard_df['Monthly_Income'].max()],
            labels=['Low Income', 'Medium-Low Income', 'Medium-High Income', 'High Income']
        )
    
    # 4. Credit Amount Categories
    if 'AMT_CREDIT' in dashboard_df.columns:
        credit_percentiles = dashboard_df['AMT_CREDIT'].quantile([0.33, 0.67])
        dashboard_df['Credit_Size'] = pd.cut(
            dashboard_df['AMT_CREDIT'],
            bins=[0, credit_percentiles[0.33], credit_percentiles[0.67], dashboard_df['AMT_CREDIT'].max()],
            labels=['Small Loan', 'Medium Loan', 'Large Loan']
        )
    
    # 5. Financial Ratios
    if all(col in dashboard_df.columns for col in ['AMT_CREDIT', 'AMT_INCOME_TOTAL']):
        dashboard_df['Credit_Income_Ratio'] = dashboard_df['AMT_CREDIT'] / dashboard_df['AMT_INCOME_TOTAL']
        dashboard_df['Debt_Burden'] = pd.cut(
            dashboard_df['Credit_Income_Ratio'],
            bins=[0, 3, 6, 10, float('inf')],
            labels=['Low Debt Burden', 'Moderate Debt Burden', 'High Debt Burden', 'Very High Debt Burden']
        )
    
    # 6. Annuity to Income Ratio
    if all(col in dashboard_df.columns for col in ['AMT_ANNUITY', 'AMT_INCOME_TOTAL']):
        dashboard_df['Annuity_Income_Ratio'] = dashboard_df['AMT_ANNUITY'] / (dashboard_df['AMT_INCOME_TOTAL'] / 12)
        dashboard_df['Payment_Burden'] = pd.cut(
            dashboard_df['Annuity_Income_Ratio'],
            bins=[0, 0.1, 0.2, 0.3, float('inf')],
            labels=['Low Payment Burden', 'Moderate Payment Burden', 'High Payment Burden', 'Very High Payment Burden']
        )
    
    # 7. Family Size Category
    if 'CNT_CHILDREN' in dashboard_df.columns:
        dashboard_df['Family_Size_Category'] = pd.cut(
            dashboard_df['CNT_CHILDREN'],
            bins=[-1, 0, 1, 2, float('inf')],
            labels=['No Children', '1 Child', '2 Children', '3+ Children']
        )
    
    # 8. External Source Score Average (if available)
    ext_source_cols = [col for col in dashboard_df.columns if col.startswith('EXT_SOURCE')]
    if ext_source_cols:
        dashboard_df['Credit_Score_Average'] = dashboard_df[ext_source_cols].mean(axis=1)
        dashboard_df['Credit_Score_Category'] = pd.cut(
            dashboard_df['Credit_Score_Average'],
            bins=[0, 0.3, 0.5, 0.7, 1.0],
            labels=['Poor Credit Score', 'Fair Credit Score', 'Good Credit Score', 'Excellent Credit Score']
        )
    
    return dashboard_df

# Apply business metrics
tableau_data = create_business_metrics(tableau_data)
print(f"✅ Created business metrics and categories")
print(f"New metrics: Age_Group, Income_Level, Credit_Size, Debt_Burden, etc.")

Creating derived business metrics...
✅ Created business metrics and categories
New metrics: Age_Group, Income_Level, Credit_Size, Debt_Burden, etc.
✅ Created business metrics and categories
New metrics: Age_Group, Income_Level, Credit_Size, Debt_Burden, etc.


In [8]:
# 5. Add prediction scores if available
print("Adding model predictions and risk scores...")

try:
    # Load prediction results
    predictions = pd.read_csv('submission_enriched_gb.csv')
    
    # Merge predictions with main data
    tableau_data = tableau_data.merge(
        predictions[['SK_ID_CURR', 'TARGET']], 
        on='SK_ID_CURR', 
        how='left',
        suffixes=('', '_Predicted')
    )
    
    # Create risk score categories
    if 'TARGET_Predicted' in tableau_data.columns:
        tableau_data['Predicted_Risk_Score'] = tableau_data['TARGET_Predicted']
        tableau_data['Risk_Score_Category'] = pd.cut(
            tableau_data['Predicted_Risk_Score'],
            bins=[0, 0.1, 0.3, 0.5, 1.0],
            labels=['Very Low Risk', 'Low Risk', 'Medium Risk', 'High Risk']
        )
        print(f"✅ Added prediction scores from enriched model")
    
except FileNotFoundError:
    print("⚠️ Prediction file not found - will create risk categories based on available data")
    # Create synthetic risk score based on available features for demo
    if all(col in tableau_data.columns for col in ['Credit_Income_Ratio', 'Age_Years']):
        # Simple risk scoring based on debt burden and age
        risk_factors = 0
        risk_factors += (tableau_data['Credit_Income_Ratio'] > 5).astype(int) * 0.3
        risk_factors += (tableau_data['Age_Years'] < 25).astype(int) * 0.2
        risk_factors += (tableau_data['Employment_Years'] < 1).astype(int) * 0.2
        
        tableau_data['Predicted_Risk_Score'] = risk_factors
        tableau_data['Risk_Score_Category'] = pd.cut(
            tableau_data['Predicted_Risk_Score'],
            bins=[-0.1, 0.2, 0.4, 0.6, 1.0],
            labels=['Very Low Risk', 'Low Risk', 'Medium Risk', 'High Risk']
        )
        print(f"✅ Created synthetic risk scores for demonstration")

print(f"Dataset now has risk scoring capabilities")

Adding model predictions and risk scores...
✅ Added prediction scores from enriched model
Dataset now has risk scoring capabilities
✅ Added prediction scores from enriched model
Dataset now has risk scoring capabilities


In [9]:
# 6. Select and organize columns for Tableau dashboard
print("Organizing columns for optimal Tableau experience...")

def select_tableau_columns(df):
    """Select and organize the most relevant columns for dashboard"""
    
    # Core identification columns
    id_cols = ['SK_ID_CURR']
    
    # Target and risk columns
    risk_cols = [
        'TARGET', 'Risk_Category', 
        'Predicted_Risk_Score', 'Risk_Score_Category'
    ]
    
    # Demographics (client-friendly)
    demo_cols = [
        'Gender', 'Age_Years', 'Age_Group',
        'Marital_Status', 'Education_Level',
        'CNT_CHILDREN', 'Family_Size_Category'
    ]
    
    # Financial information
    financial_cols = [
        'Income_Source', 'AMT_INCOME_TOTAL', 'Monthly_Income', 'Income_Level',
        'Employment_Years', 'Employment_Category',
        'Loan_Type', 'AMT_CREDIT', 'Credit_Size',
        'AMT_ANNUITY', 'Credit_Income_Ratio', 'Debt_Burden',
        'Annuity_Income_Ratio', 'Payment_Burden'
    ]
    
    # Housing and lifestyle
    lifestyle_cols = [
        'Housing_Type', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'
    ]
    
    # Credit scores (if available)
    credit_score_cols = [col for col in df.columns if col.startswith('EXT_SOURCE')]
    if 'Credit_Score_Average' in df.columns:
        credit_score_cols.extend(['Credit_Score_Average', 'Credit_Score_Category'])
    
    # Combine all relevant columns
    tableau_columns = []
    
    for col_group in [id_cols, risk_cols, demo_cols, financial_cols, lifestyle_cols, credit_score_cols]:
        for col in col_group:
            if col in df.columns:
                tableau_columns.append(col)
    
    return tableau_columns

# Select optimal columns
selected_columns = select_tableau_columns(tableau_data)
tableau_final = tableau_data[selected_columns].copy()

print(f"✅ Selected {len(selected_columns)} columns for Tableau dashboard")
print(f"Categories included:")
print(f"  • Client Demographics: Gender, Age, Family Status")
print(f"  • Financial Profile: Income, Employment, Credit")
print(f"  • Risk Assessment: Risk Categories and Scores")
print(f"  • Lifestyle Factors: Housing, Assets")

# Display column summary
print(f"\nSelected columns:")
for i, col in enumerate(selected_columns, 1):
    print(f"  {i:2d}. {col}")
    if i % 10 == 0:  # Break every 10 columns for readability
        print()

Organizing columns for optimal Tableau experience...
✅ Selected 34 columns for Tableau dashboard
Categories included:
  • Client Demographics: Gender, Age, Family Status
  • Financial Profile: Income, Employment, Credit
  • Risk Assessment: Risk Categories and Scores
  • Lifestyle Factors: Housing, Assets

Selected columns:
   1. SK_ID_CURR
   2. TARGET
   3. Risk_Category
   4. Predicted_Risk_Score
   5. Risk_Score_Category
   6. Gender
   7. Age_Years
   8. Age_Group
   9. Marital_Status
  10. Education_Level

  11. CNT_CHILDREN
  12. Family_Size_Category
  13. Income_Source
  14. AMT_INCOME_TOTAL
  15. Monthly_Income
  16. Income_Level
  17. Employment_Years
  18. Employment_Category
  19. Loan_Type
  20. AMT_CREDIT

  21. Credit_Size
  22. AMT_ANNUITY
  23. Credit_Income_Ratio
  24. Debt_Burden
  25. Annuity_Income_Ratio
  26. Payment_Burden
  27. Housing_Type
  28. FLAG_OWN_CAR
  29. FLAG_OWN_REALTY
  30. EXT_SOURCE_1

  31. EXT_SOURCE_2
  32. EXT_SOURCE_3
  33. Credit_Score_Avera

In [12]:
# 7. Clean and optimize data for Tableau
print("Final data cleaning and optimization...")

def optimize_for_tableau(df):
    """Final optimizations for Tableau performance"""
    
    optimized_df = df.copy()
    
    # 1. Handle missing values with meaningful labels
    for col in optimized_df.columns:
        if optimized_df[col].dtype == 'object':
            optimized_df[col] = optimized_df[col].fillna('Not Specified')
        elif optimized_df[col].dtype.name == 'category':
            # For categorical columns, convert to string first, then fill NAs
            optimized_df[col] = optimized_df[col].astype(str)
            optimized_df[col] = optimized_df[col].replace('nan', 'Not Specified')
            optimized_df[col] = optimized_df[col].fillna('Not Specified')
        elif optimized_df[col].dtype in ['float64', 'int64']:
            if 'AMT_' in col or 'Income' in col or 'Credit' in col:
                optimized_df[col] = optimized_df[col].fillna(0)
            else:
                optimized_df[col] = optimized_df[col].fillna(optimized_df[col].median())
    
    # 2. Round financial amounts for cleaner display (only for numeric columns)
    financial_columns = [col for col in optimized_df.columns if 'AMT_' in col]
    for col in financial_columns:
        if col in optimized_df.columns and optimized_df[col].dtype in ['float64', 'int64']:
            optimized_df[col] = optimized_df[col].round(0).astype('int64')
    
    # 3. Round numeric income columns
    if 'Monthly_Income' in optimized_df.columns and optimized_df['Monthly_Income'].dtype in ['float64', 'int64']:
        optimized_df['Monthly_Income'] = optimized_df['Monthly_Income'].round(0).astype('int64')
    
    # 4. Round ratios and scores
    ratio_columns = [col for col in optimized_df.columns if 'Ratio' in col or 'Score' in col]
    for col in ratio_columns:
        if col in optimized_df.columns and optimized_df[col].dtype in ['float64']:
            optimized_df[col] = optimized_df[col].round(3)
    
    # 5. Convert boolean flags to readable text
    flag_columns = [col for col in optimized_df.columns if col.startswith('FLAG_')]
    for col in flag_columns:
        if col in optimized_df.columns:
            optimized_df[col] = optimized_df[col].map({1: 'Yes', 0: 'No'}).fillna('Unknown')
    
    # 6. Round age and employment years
    if 'Age_Years' in optimized_df.columns and optimized_df['Age_Years'].dtype in ['float64']:
        optimized_df['Age_Years'] = optimized_df['Age_Years'].round(1)
    
    if 'Employment_Years' in optimized_df.columns and optimized_df['Employment_Years'].dtype in ['float64']:
        optimized_df['Employment_Years'] = optimized_df['Employment_Years'].round(1)
    
    # 7. Ensure string columns for categorical data (convert after processing)
    categorical_columns = [col for col in optimized_df.columns 
                          if ('_Group' in col or '_Level' in col or '_Category' in col or 
                              '_Status' in col or '_Type' in col or '_Source' in col or 
                              '_Burden' in col) and col != 'SK_ID_CURR']
    for col in categorical_columns:
        if col in optimized_df.columns:
            optimized_df[col] = optimized_df[col].astype(str)
    
    return optimized_df

# Apply final optimizations
tableau_final = optimize_for_tableau(tableau_final)

# Data quality check
print(f"✅ Final dataset optimized for Tableau")
print(f"\nDataset Summary:")
print(f"  • Total records: {len(tableau_final):,}")
print(f"  • Total columns: {len(tableau_final.columns)}")
print(f"  • Missing values: {tableau_final.isnull().sum().sum()}")
print(f"  • File size estimate: ~{(tableau_final.memory_usage(deep=True).sum() / 1024**2):.1f} MB")

# Show data types
print(f"\nColumn Types:")
print(f"  • String/Object: {len(tableau_final.select_dtypes(include=['object']).columns)}")
print(f"  • Numerical: {len(tableau_final.select_dtypes(include=['int64', 'float64']).columns)}")

# Sample of the data
print(f"\nSample of final dataset:")
print(tableau_final.head(3).to_string())

Final data cleaning and optimization...
✅ Final dataset optimized for Tableau

Dataset Summary:
  • Total records: 307,511
  • Total columns: 34
  • Missing values: 307511
✅ Final dataset optimized for Tableau

Dataset Summary:
  • Total records: 307,511
  • Total columns: 34
  • Missing values: 307511
  • File size estimate: ~400.1 MB

Column Types:
  • String/Object: 18
  • Numerical: 16

Sample of final dataset:
   SK_ID_CURR  TARGET               Risk_Category  Predicted_Risk_Score Risk_Score_Category  Gender  Age_Years Age_Group Marital_Status Education_Level  CNT_CHILDREN Family_Size_Category        Income_Source  AMT_INCOME_TOTAL  Monthly_Income        Income_Level  Employment_Years Employment_Category         Loan_Type  AMT_CREDIT  Credit_Size  AMT_ANNUITY  Credit_Income_Ratio           Debt_Burden  Annuity_Income_Ratio            Payment_Burden Housing_Type FLAG_OWN_CAR FLAG_OWN_REALTY  EXT_SOURCE_1  EXT_SOURCE_2  EXT_SOURCE_3  Credit_Score_Average Credit_Score_Category
0     

In [13]:
# 8. Save the Tableau-optimized dataset
print("Saving Tableau-optimized dataset...")

# Create the final CSV file
output_filename = 'credit_risk_dashboard_data.csv'
tableau_final.to_csv(output_filename, index=False)

print(f"✅ Saved Tableau dataset: {output_filename}")
print(f"File size: {(tableau_final.memory_usage(deep=True).sum() / 1024**2):.1f} MB")

# Create a data dictionary for the dashboard
data_dict = []

column_descriptions = {
    'SK_ID_CURR': 'Unique client identifier',
    'TARGET': 'Actual default status (1=Default, 0=No Default)',
    'Risk_Category': 'Client risk classification',
    'Predicted_Risk_Score': 'Model-predicted default probability',
    'Risk_Score_Category': 'Risk level based on prediction',
    'Gender': 'Client gender',
    'Age_Years': 'Client age in years',
    'Age_Group': 'Age category grouping',
    'Marital_Status': 'Family status',
    'Education_Level': 'Highest education completed',
    'CNT_CHILDREN': 'Number of children',
    'Family_Size_Category': 'Family size grouping',
    'Income_Source': 'Type of employment/income',
    'AMT_INCOME_TOTAL': 'Total annual income',
    'Monthly_Income': 'Monthly income amount',
    'Income_Level': 'Income category',
    'Employment_Years': 'Years in current employment',
    'Employment_Category': 'Employment duration group',
    'Loan_Type': 'Type of credit product',
    'AMT_CREDIT': 'Total credit amount',
    'Credit_Size': 'Loan size category',
    'AMT_ANNUITY': 'Monthly payment amount',
    'Credit_Income_Ratio': 'Credit amount / Income ratio',
    'Debt_Burden': 'Debt burden level',
    'Annuity_Income_Ratio': 'Payment / Income ratio',
    'Payment_Burden': 'Payment burden level',
    'Housing_Type': 'Living situation',
    'FLAG_OWN_CAR': 'Car ownership (Yes/No)',
    'FLAG_OWN_REALTY': 'Property ownership (Yes/No)',
    'Credit_Score_Average': 'Average external credit score',
    'Credit_Score_Category': 'Credit score level'
}

for col in tableau_final.columns:
    data_dict.append({
        'Column_Name': col,
        'Description': column_descriptions.get(col, 'See data documentation'),
        'Data_Type': str(tableau_final[col].dtype),
        'Sample_Values': str(tableau_final[col].dropna().head(3).tolist()[:3])
    })

# Save data dictionary
dict_df = pd.DataFrame(data_dict)
dict_filename = 'credit_risk_data_dictionary.csv'
dict_df.to_csv(dict_filename, index=False)

print(f"✅ Saved data dictionary: {dict_filename}")
print(f"\nData Dictionary Preview:")
print(dict_df.head(10).to_string(index=False))

Saving Tableau-optimized dataset...
✅ Saved Tableau dataset: credit_risk_dashboard_data.csv
✅ Saved Tableau dataset: credit_risk_dashboard_data.csv
File size: 400.1 MB
✅ Saved data dictionary: credit_risk_data_dictionary.csv

Data Dictionary Preview:
         Column_Name                                     Description Data_Type                                                                      Sample_Values
          SK_ID_CURR                        Unique client identifier     int64                                                           [100002, 100003, 100004]
              TARGET Actual default status (1=Default, 0=No Default)     int64                                                                          [1, 0, 0]
       Risk_Category                      Client risk classification    object ['High Risk (Payment Issues)', 'Low Risk (Good Client)', 'Low Risk (Good Client)']
Predicted_Risk_Score             Model-predicted default probability   float64                       

## 📊 Tableau Dashboard Recommendations

### Key Visualizations to Create:

#### 1. **Executive Summary Dashboard**
- **Risk Overview KPIs**: Total clients, high-risk percentage, average risk score
- **Portfolio Summary**: Total credit amount, average loan size, default rate
- **Geographic Distribution**: Risk by region (if location data available)

#### 2. **Client Demographics Analysis**
- **Age Distribution**: Age groups vs. risk levels (bar chart)
- **Income Analysis**: Income levels vs. default rates (scatter plot)
- **Education Impact**: Education level vs. risk scores (stacked bar)
- **Family Status**: Marital status and children vs. risk (heatmap)

#### 3. **Financial Risk Analysis**
- **Debt Burden Analysis**: Credit-to-income ratios by risk category
- **Payment Burden**: Monthly payment vs. income scatter plot
- **Loan Size Impact**: Credit amount vs. default probability
- **Employment Stability**: Employment years vs. risk levels

#### 4. **Predictive Risk Dashboard**
- **Risk Score Distribution**: Histogram of predicted risk scores
- **Risk vs. Reality**: Predicted vs. actual default rates
- **Model Performance**: ROC curve and confusion matrix
- **High-Risk Client Profiles**: Characteristics of high-risk segments

### Color Coding Recommendations:
- **Green**: Low risk, good performance
- **Yellow/Orange**: Medium risk, caution
- **Red**: High risk, attention needed
- **Blue**: Neutral/informational metrics

### Interactive Filters to Add:
- Age Group slider
- Income Level dropdown
- Loan Type selection
- Risk Category filter
- Employment Category filter

### Dashboard Navigation:
1. Start with Executive Summary
2. Drill down to Demographics
3. Analyze Financial Patterns
4. Review Risk Predictions
5. Action Items and Insights

In [14]:
# 9. Create sample visualizations for validation
print("Creating sample analysis to validate data quality...")

# Basic statistics for dashboard validation
print("\n📊 DATASET VALIDATION FOR TABLEAU")
print("="*60)

# 1. Risk distribution
if 'Risk_Category' in tableau_final.columns:
    risk_dist = tableau_final['Risk_Category'].value_counts()
    print(f"\n🎯 Risk Distribution:")
    for category, count in risk_dist.items():
        percentage = (count / len(tableau_final)) * 100
        print(f"  {category}: {count:,} ({percentage:.1f}%)")

# 2. Age analysis
if 'Age_Group' in tableau_final.columns:
    age_dist = tableau_final['Age_Group'].value_counts().sort_index()
    print(f"\n👥 Age Distribution:")
    for age_group, count in age_dist.items():
        percentage = (count / len(tableau_final)) * 100
        print(f"  {age_group}: {count:,} ({percentage:.1f}%)")

# 3. Income analysis
if 'Income_Level' in tableau_final.columns:
    income_dist = tableau_final['Income_Level'].value_counts()
    print(f"\n💰 Income Distribution:")
    for income_level, count in income_dist.items():
        percentage = (count / len(tableau_final)) * 100
        print(f"  {income_level}: {count:,} ({percentage:.1f}%)")

# 4. Loan type analysis
if 'Loan_Type' in tableau_final.columns:
    loan_dist = tableau_final['Loan_Type'].value_counts()
    print(f"\n🏦 Loan Type Distribution:")
    for loan_type, count in loan_dist.items():
        percentage = (count / len(tableau_final)) * 100
        print(f"  {loan_type}: {count:,} ({percentage:.1f}%)")

# 5. Key financial metrics
financial_metrics = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY']
print(f"\n💵 Financial Metrics Summary:")
for metric in financial_metrics:
    if metric in tableau_final.columns:
        mean_val = tableau_final[metric].mean()
        median_val = tableau_final[metric].median()
        print(f"  {metric}:")
        print(f"    Mean: ${mean_val:,.0f}")
        print(f"    Median: ${median_val:,.0f}")

print(f"\n✅ Data validation complete!")
print(f"Dataset ready for Tableau import: {output_filename}")
print(f"Supporting documentation: {dict_filename}")

# Quick quality checks
null_counts = tableau_final.isnull().sum()
if null_counts.sum() > 0:
    print(f"\n⚠️ Columns with missing values:")
    for col, nulls in null_counts[null_counts > 0].items():
        print(f"  {col}: {nulls:,} missing")
else:
    print(f"\n✅ No missing values - dataset is clean!")

Creating sample analysis to validate data quality...

📊 DATASET VALIDATION FOR TABLEAU

🎯 Risk Distribution:
  Low Risk (Good Client): 282,686 (91.9%)
  High Risk (Payment Issues): 24,825 (8.1%)

👥 Age Distribution:
  25-34: 72,429 (23.6%)
  35-44: 84,261 (27.4%)
  45-54: 70,190 (22.8%)
  55-64: 60,522 (19.7%)
  65+: 7,876 (2.6%)
  Under 25: 12,233 (4.0%)

💰 Income Distribution:
  Low Income: 100,578 (32.7%)
  Medium-High Income: 82,213 (26.7%)
  High Income: 71,538 (23.3%)
  Medium-Low Income: 53,182 (17.3%)

🏦 Loan Type Distribution:
  Cash Loan: 278,232 (90.5%)
  Revolving Credit: 29,279 (9.5%)

💵 Financial Metrics Summary:
  AMT_INCOME_TOTAL:
    Mean: $168,798
    Median: $147,150
  AMT_CREDIT:
    Mean: $599,026
    Median: $513,531
  AMT_ANNUITY:
    Mean: $27,108
    Median: $24,903

✅ Data validation complete!
Dataset ready for Tableau import: credit_risk_dashboard_data.csv
Supporting documentation: credit_risk_data_dictionary.csv

⚠️ Columns with missing values:
  Predicted_R

## 🚀 Next Steps: Building Your Tableau Dashboard

### 1. **Import the Data**
- Open Tableau Desktop
- Connect to `credit_risk_dashboard_data.csv`
- Verify data types (Tableau should auto-detect most correctly)

### 2. **Dashboard Structure**
```
📊 CREDIT RISK EXECUTIVE DASHBOARD
├── 📈 Executive Summary (Top Row)
│   ├── Total Clients KPI
│   ├── High Risk % KPI  
│   ├── Average Risk Score KPI
│   └── Total Portfolio Value KPI
├── 👥 Demographics Analysis (Middle Left)
│   ├── Age Group vs Risk (Bar Chart)
│   └── Income Level Distribution (Pie Chart)
├── 💰 Financial Analysis (Middle Right)
│   ├── Debt Burden Heatmap
│   └── Credit Size vs Risk (Scatter Plot)
└── 🎯 Risk Insights (Bottom Row)
    ├── Risk Score Distribution (Histogram)
    └── High-Risk Client Characteristics (Table)
```

### 3. **Key Calculated Fields to Create**
```
// Risk Rate by Category
SUM([TARGET]) / COUNT([SK_ID_CURR])

// Average Risk Score
AVG([Predicted_Risk_Score])

// Portfolio Value
SUM([AMT_CREDIT])

// High Risk Indicator
IF [Risk_Score_Category] = "High Risk" THEN 1 ELSE 0 END
```

### 4. **Color Palette**
- **🔴 High Risk**: #D32F2F (Red)
- **🟠 Medium Risk**: #FF9800 (Orange) 
- **🟡 Low Risk**: #FBC02D (Yellow)
- **🟢 Very Low Risk**: #388E3C (Green)

### 5. **Interactive Elements**
- **Filters**: Age Group, Income Level, Loan Type
- **Parameters**: Risk Threshold Slider
- **Actions**: Click demographics to filter financial charts

Run the notebook cells above to generate your optimized dataset!

In [15]:
# 10. Final summary and file verification
print("🎉 TABLEAU DATASET CREATION COMPLETED!")
print("="*60)
print("✅ Successfully created optimized dataset for Tableau dashboard")
print(f"📁 Main dataset: credit_risk_dashboard_data.csv")
print(f"📖 Documentation: credit_risk_data_dictionary.csv")
print("\n🚀 Ready for Tableau import!")
print("\nNext steps:")
print("1. Open Tableau Desktop")
print("2. Connect to 'credit_risk_dashboard_data.csv'")
print("3. Follow the dashboard recommendations in the markdown cells above")
print("4. Create visualizations using the client-friendly column names")
print("\n💡 The dataset includes:")
print("   • 307,511 client records")
print("   • 34 optimized columns")
print("   • Client-friendly labels and categories")
print("   • Risk scores and business metrics")
print("   • Clean data ready for visualization")

🎉 TABLEAU DATASET CREATION COMPLETED!
✅ Successfully created optimized dataset for Tableau dashboard
📁 Main dataset: credit_risk_dashboard_data.csv
📖 Documentation: credit_risk_data_dictionary.csv

🚀 Ready for Tableau import!

Next steps:
1. Open Tableau Desktop
2. Connect to 'credit_risk_dashboard_data.csv'
3. Follow the dashboard recommendations in the markdown cells above
4. Create visualizations using the client-friendly column names

💡 The dataset includes:
   • 307,511 client records
   • 34 optimized columns
   • Client-friendly labels and categories
   • Risk scores and business metrics
   • Clean data ready for visualization


In [16]:
# Display the columns in the tableau_final dataset
print("Columns in the Tableau dataset:")
print("="*50)
for i, col in enumerate(tableau_final.columns, 1):
    print(f"{i:2d}. {col}")
    
print(f"\nTotal columns: {len(tableau_final.columns)}")
print(f"Dataset shape: {tableau_final.shape}")

Columns in the Tableau dataset:
 1. SK_ID_CURR
 2. TARGET
 3. Risk_Category
 4. Predicted_Risk_Score
 5. Risk_Score_Category
 6. Gender
 7. Age_Years
 8. Age_Group
 9. Marital_Status
10. Education_Level
11. CNT_CHILDREN
12. Family_Size_Category
13. Income_Source
14. AMT_INCOME_TOTAL
15. Monthly_Income
16. Income_Level
17. Employment_Years
18. Employment_Category
19. Loan_Type
20. AMT_CREDIT
21. Credit_Size
22. AMT_ANNUITY
23. Credit_Income_Ratio
24. Debt_Burden
25. Annuity_Income_Ratio
26. Payment_Burden
27. Housing_Type
28. FLAG_OWN_CAR
29. FLAG_OWN_REALTY
30. EXT_SOURCE_1
31. EXT_SOURCE_2
32. EXT_SOURCE_3
33. Credit_Score_Average
34. Credit_Score_Category

Total columns: 34
Dataset shape: (307511, 34)


In [2]:
# Load the saved Tableau dataset for analysis
import pandas as pd
import numpy as np

print("="*60)
print("DATA QUALITY ANALYSIS FOR TABLEAU")
print("="*60)

# Load the saved Tableau dataset
try:
    tableau_final = pd.read_csv('credit_risk_dashboard_data.csv')
    print(f"✅ Loaded Tableau dataset: {tableau_final.shape}")
except FileNotFoundError:
    print("❌ Tableau dataset file not found. Please run the previous cells first.")
    
# 1. Check for null values in key columns
print("\n1. NULL VALUES ANALYSIS:")
print("-" * 30)
key_columns = ['TARGET', 'Risk_Category', 'Predicted_Risk_Score', 'Risk_Score_Category']
for col in key_columns:
    if col in tableau_final.columns:
        null_count = tableau_final[col].isnull().sum()
        null_pct = (null_count / len(tableau_final)) * 100
        print(f"{col:20}: {null_count:,} nulls ({null_pct:.2f}%)")
    else:
        print(f"{col:20}: Column not found!")

# 2. Check data types
print("\n2. DATA TYPES:")
print("-" * 30)
for col in key_columns:
    if col in tableau_final.columns:
        print(f"{col:20}: {tableau_final[col].dtype}")

# 3. Check unique values in categorical columns
print("\n3. CATEGORICAL VALUES:")
print("-" * 30)
if 'Risk_Category' in tableau_final.columns:
    print(f"Risk_Category values: {tableau_final['Risk_Category'].unique()}")
if 'Risk_Score_Category' in tableau_final.columns:
    print(f"Risk_Score_Category values: {tableau_final['Risk_Score_Category'].unique()}")

# 4. Check Predicted_Risk_Score statistics
print("\n4. PREDICTED RISK SCORE ANALYSIS:")
print("-" * 30)
if 'Predicted_Risk_Score' in tableau_final.columns:
    if tableau_final['Predicted_Risk_Score'].isnull().all():
        print("❌ ALL Predicted_Risk_Score values are NULL!")
        print("This suggests the prediction model didn't run or failed to save predictions.")
    else:
        print(f"Min: {tableau_final['Predicted_Risk_Score'].min():.4f}")
        print(f"Max: {tableau_final['Predicted_Risk_Score'].max():.4f}")
        print(f"Mean: {tableau_final['Predicted_Risk_Score'].mean():.4f}")
        print(f"Median: {tableau_final['Predicted_Risk_Score'].median():.4f}")
        
        # Show distribution of risk scores
        print(f"\nRisk Score Distribution:")
        print(tableau_final['Predicted_Risk_Score'].describe())
else:
    print("❌ Predicted_Risk_Score column not found!")

# 5. Sample some records to see the data
print("\n5. SAMPLE DATA:")
print("-" * 30)
if len(tableau_final) > 0:
    sample_cols = ['SK_ID_CURR', 'TARGET', 'Risk_Category', 'Predicted_Risk_Score', 'Risk_Score_Category']
    available_cols = [col for col in sample_cols if col in tableau_final.columns]
    print(tableau_final[available_cols].head(10))

DATA QUALITY ANALYSIS FOR TABLEAU
✅ Loaded Tableau dataset: (307511, 34)

1. NULL VALUES ANALYSIS:
------------------------------
TARGET              : 0 nulls (0.00%)
Risk_Category       : 0 nulls (0.00%)
Predicted_Risk_Score: 307,511 nulls (100.00%)
Risk_Score_Category : 0 nulls (0.00%)

2. DATA TYPES:
------------------------------
TARGET              : int64
Risk_Category       : object
Predicted_Risk_Score: float64
Risk_Score_Category : object

3. CATEGORICAL VALUES:
------------------------------
Risk_Category values: ['High Risk (Payment Issues)' 'Low Risk (Good Client)']
Risk_Score_Category values: ['Not Specified']

4. PREDICTED RISK SCORE ANALYSIS:
------------------------------
❌ ALL Predicted_Risk_Score values are NULL!
This suggests the prediction model didn't run or failed to save predictions.

5. SAMPLE DATA:
------------------------------
   SK_ID_CURR  TARGET               Risk_Category  Predicted_Risk_Score  \
0      100002       1  High Risk (Payment Issues)         

In [3]:
# Fix the missing predictions by generating them properly
print("="*60)
print("GENERATING MISSING PREDICTIONS")
print("="*60)

# Check if we have enriched data available
print("Loading enriched dataset for prediction generation...")

try:
    # Load the enriched training data
    app_train = pd.read_csv('application_train_enriched.csv', encoding='utf-8')
    print(f"✅ Loaded enriched data: {app_train.shape}")
except:
    try:
        app_train = pd.read_csv('application_train_enriched.csv', encoding='latin-1')
        print(f"✅ Loaded enriched data with latin-1: {app_train.shape}")
    except:
        app_train = pd.read_csv('application_train_enriched.csv', encoding='cp1252')
        print(f"✅ Loaded enriched data with cp1252: {app_train.shape}")

# Quick model training to generate predictions
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("\nTraining quick model for predictions...")

# Prepare the data
X = app_train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = app_train['TARGET']

# Handle categorical variables
label_encoders = {}
for col in X.select_dtypes(include=['object']).columns:
    if X[col].dtype == 'object':  # Double check
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le

# Fill missing values
X = X.fillna(X.median())

print(f"Features prepared: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Train a quick model
model = GradientBoostingClassifier(n_estimators=50, max_depth=4, random_state=42)
model.fit(X, y)

# Generate predictions (probabilities)
predictions_proba = model.predict_proba(X)[:, 1]  # Probability of class 1 (high risk)

print(f"\n✅ Generated {len(predictions_proba)} predictions")
print(f"Prediction range: {predictions_proba.min():.4f} to {predictions_proba.max():.4f}")
print(f"Mean prediction: {predictions_proba.mean():.4f}")

# Add predictions to our dataset
tableau_final['Predicted_Risk_Score'] = predictions_proba

# Create risk score categories based on predictions
def categorize_risk_score(score):
    if score < 0.3:
        return "Low Risk (0.0-0.3)"
    elif score < 0.6:
        return "Medium Risk (0.3-0.6)"
    else:
        return "High Risk (0.6+)"

tableau_final['Risk_Score_Category'] = tableau_final['Predicted_Risk_Score'].apply(categorize_risk_score)

print("\n✅ Updated tableau dataset with predictions")
print(f"Risk Score Categories distribution:")
print(tableau_final['Risk_Score_Category'].value_counts())

GENERATING MISSING PREDICTIONS
Loading enriched dataset for prediction generation...
✅ Loaded enriched data: (307511, 160)

Training quick model for predictions...
Features prepared: (307511, 158)
Target distribution: {0: 282686, 1: 24825}

✅ Generated 307511 predictions
Prediction range: 0.0136 to 0.8231
Mean prediction: 0.0808

✅ Updated tableau dataset with predictions
Risk Score Categories distribution:
Risk_Score_Category
Low Risk (0.0-0.3)       301357
Medium Risk (0.3-0.6)      6046
High Risk (0.6+)            108
Name: count, dtype: int64


In [4]:
# Verify the fixed dataset and calculate Tableau metrics
print("="*60)
print("TABLEAU ANALYSIS VERIFICATION")
print("="*60)

# 1. Verify no more null predictions
print("1. NULL VALUES CHECK:")
print("-" * 30)
key_columns = ['TARGET', 'Risk_Category', 'Predicted_Risk_Score', 'Risk_Score_Category']
for col in key_columns:
    null_count = tableau_final[col].isnull().sum()
    null_pct = (null_count / len(tableau_final)) * 100
    print(f"{col:20}: {null_count:,} nulls ({null_pct:.2f}%)")

# 2. Calculate Tableau Dashboard Metrics
print("\n2. TABLEAU DASHBOARD METRICS:")
print("-" * 30)

# Average Predicted Risk Score
avg_risk_score = tableau_final['Predicted_Risk_Score'].mean()
print(f"📊 Average Predicted Risk Score: {avg_risk_score:.4f}")

# High-Risk Share (%)
high_risk_count = len(tableau_final[tableau_final['Risk_Category'].str.contains('High Risk')])
total_clients = len(tableau_final)
high_risk_share = (high_risk_count / total_clients) * 100
print(f"📊 High-Risk Share: {high_risk_share:.2f}% ({high_risk_count:,} out of {total_clients:,} clients)")

# Risk distribution by actual TARGET
print(f"\n3. ACTUAL vs PREDICTED RISK:")
print("-" * 30)
risk_comparison = pd.crosstab(tableau_final['TARGET'], tableau_final['Risk_Score_Category'])
print(risk_comparison)

# 4. Key statistics for Tableau
print(f"\n4. KEY STATISTICS FOR TABLEAU:")
print("-" * 30)
print(f"• Total Clients: {total_clients:,}")
print(f"• Actual Defaulters: {tableau_final['TARGET'].sum():,} ({(tableau_final['TARGET'].mean()*100):.1f}%)")
print(f"• Predicted High Risk (0.6+): {len(tableau_final[tableau_final['Predicted_Risk_Score'] >= 0.6]):,}")
print(f"• Predicted Medium Risk (0.3-0.6): {len(tableau_final[(tableau_final['Predicted_Risk_Score'] >= 0.3) & (tableau_final['Predicted_Risk_Score'] < 0.6)]):,}")
print(f"• Predicted Low Risk (<0.3): {len(tableau_final[tableau_final['Predicted_Risk_Score'] < 0.3]):,}")

# 5. Sample of fixed data
print(f"\n5. SAMPLE OF CORRECTED DATA:")
print("-" * 30)
sample_cols = ['SK_ID_CURR', 'TARGET', 'Risk_Category', 'Predicted_Risk_Score', 'Risk_Score_Category']
print(tableau_final[sample_cols].head(10))

# Save the corrected dataset
tableau_final.to_csv('credit_risk_dashboard_data_fixed.csv', index=False)
print(f"\n✅ Saved corrected dataset: credit_risk_dashboard_data_fixed.csv")
print(f"Dataset shape: {tableau_final.shape}")
print("\n🎯 READY FOR TABLEAU ANALYSIS! 🎯")

TABLEAU ANALYSIS VERIFICATION
1. NULL VALUES CHECK:
------------------------------
TARGET              : 0 nulls (0.00%)
Risk_Category       : 0 nulls (0.00%)
Predicted_Risk_Score: 0 nulls (0.00%)
Risk_Score_Category : 0 nulls (0.00%)

2. TABLEAU DASHBOARD METRICS:
------------------------------
📊 Average Predicted Risk Score: 0.0808
📊 High-Risk Share: 8.07% (24,825 out of 307,511 clients)

3. ACTUAL vs PREDICTED RISK:
------------------------------
Risk_Score_Category  High Risk (0.6+)  Low Risk (0.0-0.3)  \
TARGET                                                      
0                                  23              279293   
1                                  85               22064   

Risk_Score_Category  Medium Risk (0.3-0.6)  
TARGET                                      
0                                     3370  
1                                     2676  

4. KEY STATISTICS FOR TABLEAU:
------------------------------
• Total Clients: 307,511
• Actual Defaulters: 24,825 (8.1%